<a href="https://colab.research.google.com/github/minhng/notebooks/blob/master/text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import xgboost
from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
data = pd.read_csv('https://s3-us-west-1.amazonaws.com/ra-training/ra_data_classifier.csv', encoding = "ISO-8859-1")
data.head(5)

,hid,chunk,has_space
0,33BFF6QPI1YEFYISIWWDVQKGH8RW3Z,"Landmark Center, 8th Fl",0
1,3HUR21WDDUCUK1K6HMLPN3U0ZBSYX0,Contact: The C3 team at MakemeC3@cic.us -- Add...,0
2,3566S7OX5D63FG3CNKAIFH62QX517B,"A powerful tool for developers, the MySQL Data...",0
3,3GV1I4SEO9CX1NTBXKN9TIFKTSQ6LN,"Easy access to T, Hubway, and parking",0
4,37SDSEDIN9P7FU8VXP2OTH2X1HB18Y,Check out our Private Offices,1


Data preprocessing - drop 'hid' column + remove duplicates in 'chunk' column (if there is).

In [0]:
data_single = data.drop(['hid',], axis = 1)
data_single = data_sel.drop_duplicates('chunk')
data_single.shape

(100, 2)

Transform all text to lower case and remove any accents from letters.

In [0]:
data_single['chunk'] = data_single['chunk'].str.lower()
def unidecode_text(text):
    try:
        text = unidecode.unidecode(text)
    except:
        pass
    return text
data_single['chunk'] = data_single.apply(lambda row: unidecode_text(row['chunk']), axis=1)
data_single.head(5)

,chunk,has_space
0,"landmark center, 8th fl",0
1,contact: the c3 team at makemec3@cic.us -- add...,0
2,"a powerful tool for developers, the mysql data...",0
3,"easy access to t, hubway, and parking",0
4,check out our private offices,1


We will split the dataset into training and testing sets so that we can train and test classifier. <br>
Training dataset = 75% and testing dataset = 25% <br>
We will also encode our target column so that it can be used in machine learning models.

In [0]:
# split the dataset into training and testing datasets 
train_x, test_x, train_y, test_y = model_selection.train_test_split(data_single['chunk'], data_single['has_space'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)
train_y.shape

(75,)

Word Level TF-IDF : Matrix representing tf-idf scores of every term in different documents. <br>
Vectorization is a method that counts the occurance of a word in one sample then compares it with the occurance of the same word over all the samples. It repeats this process for each word and outputs a vector of numbers, where each number represents a word.

In [0]:
# tf-idf (1-gram)
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data_single['chunk'])
train_x_tfidf =  tfidf_vect.transform(train_x)
test_x_tfidf =  tfidf_vect.transform(test_x)

Classification Model: XGBoost - a gradient boosted decision tree classifier. <br>
Run our trained labels against testing labels yields 72% in accuracy. 

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)    
    return metrics.accuracy_score(predictions, test_y)

# XGBoost on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), train_x_tfidf.tocsc(), train_y, test_x_tfidf.tocsc())
print("Result accuracy: ", accuracy)

Result accuracy:  0.72
